In [1]:
import sys
# This is a workaround for running the script from the parent directory
sys.path.append('../')

# For loading sub file
import pysubs2

# For printing
import pprint

from langchain_openai import ChatOpenAI

In [2]:
import json
json_file = open("../config.json")
json_data = json.load(json_file)
api_key = json_data['openai_api_key']

## Scene Structure

In [3]:
ass_pass = "../sample/sample4.ass"
subs = pysubs2.load(ass_pass)
lines = "\n".join(str(line.text) for line in subs)
lines

'（誘導者の残響）招かれもせずに\\Nワシの玉座に入ったな\n（スロアン）ありえない…\n（誘導者の残響）ワシは深淵から蘇りし邪神\n最高誘導者だ\n（スロアン）残響だ！\nオリックスの記憶と繋がっているのか？\n（シヴ・アラス）兄よ！\n王よ！\n我が誘導者よ！\n戻ってこられたのか！\n（誘導者の残響）シヴ・アラスか\n我々の戦争はどうなったのだ？\n（シヴ・アラス）あなたは空の戦士に殺された\nそして、あなたの目前にいるのが\\Nその張本人だ！\n（誘導者の残響）このワシが殺されただと？\n（サバスン）残念ながら、噓ではない\nお前は死んだ神の記憶に過ぎない\nついになりたがっていた剣になることが\\N出来たようだな\n（誘導者の残響）ワシはアッカの死をもって即位した\n虫の亡骸から\\N玉座を造ったばかりだった\n（シヴ・アラス）あなたの殺害者を殺せば\nあなたの肉体は蘇る\nあなたの玉座は復活するのだ\n（誘導者の残響）破壊されたのであれば\nワシは滅するべきなのだ\nワシの継承者であり、\\Nワシを殺した者よ\n進み出るのだ\n（サバスン）二人とも大人げないぞ\n私たちはついに\\N自らの道を切り開くことができる\n（シヴ・アラス）ダメだ\n我々を再び見捨てることは許さない！\n（スロアン）こんなことは…ありえない！\n一度合流しよう\n早くそこから出るんだ！\n（シヴ・アラス）オリックス、私があなたの仇を討つ\n（誘導者の残響）刃を研ぐのだ\nその噓が\\Nワシの論理の真実となるように！\n（スロアン）行け\n（シヴ・アラス）なぜ兄は再び肉と骨の姿となり\n剣を手に持つことを拒むのだ？\n（誘導者の残響）ワシは深淵と交信した\nもしワシが本当に死んだのなら\nそれまでのことだ\nアイアット\nお前は欺瞞者に導かれるままに\n異端に堕ち\n鋭さを失った\nそんなお前が\\Nどんな真実を語ることができる？\n（シヴ・アラス）不相応な話し手は罰せられ\n偽りの偶像神は失墜し\n我は玉座を失った\nそしてあなたは死んだ\n我らの背後にあるものは\\N全て穢れている\n真実はこの先にある\n捧げもの\n剣\nそして我が王、オリックスが\nあなたはここに、\\N我と共にいる\nこれで我々は正しい道筋に\\N戻ることができる\n共に制服しようぞ\n（誘導者の残響）な

In [4]:
from src.logic.context import determine_scene_structure

llm = ChatOpenAI(model="gpt-4o", temperature=json_data['temperature'], api_key=api_key)

In [5]:
format_results = determine_scene_structure(llm, 'ja', 'en', lines)
pprint.pp(format_results)

('**Scene Type**: Interactive Dialogue  \n'
 '**Speaker Count**: 3+  \n'
 '**Interaction Style**: The characters are actively engaging with each other, '
 "responding to each other's statements and questions. There are clear "
 'exchanges between the characters, indicating a dynamic interaction rather '
 'than isolated monologues.  \n'
 '**Delivery Format**: Speaker-separated entries  \n'
 '**Notes**: The dialogue involves multiple characters, including the '
 "Inducer's Echo, Sloane, Shiv Arath, Savathun, and Eris Morn. The characters "
 'discuss themes of power, betrayal, and identity, often addressing each other '
 'directly and reacting to previous statements. The structure suggests a '
 'complex narrative with shifting alliances and confrontations.')


## Web Search

In [6]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from src.logic.context import gather_context_from_web

In [7]:
import os
os.environ["TAVILY_API_KEY"] = json_data["tavily_api_key"]
search_tool = TavilySearchResults(k=3)

In [8]:
web_search_results = gather_context_from_web(llm, search_tool, output_lang="en",
                                             series_name="destiny 2", keywords="episode heresy", transcript=lines)

pprint.pp(web_search_results)

('**Character Relationships:** The story involves three major Hive Gods: Oryx, '
 'Xivu Arath, and Savathun, who are central to the narrative. Eris is another '
 'key character whose actions against Oryx lead to significant conflict, and '
 'there is a potential alliance forming between Xivu Arath and Savathun.\n'
 '\n'
 '**Story Premise and Genre:** Destiny 2 is set in a science fiction universe '
 'where players navigate through complex narratives involving gods and '
 'mythical entities. The story in the Heresy acts focuses on themes of '
 'betrayal, power struggles, and the uncovering of hidden truths about The '
 'Hive and their connection to The Light.')


## Character Identification

In [9]:
from src.logic.context import identify_characters

In [10]:
characters_output = identify_characters(llm, input_lang="ja", output_lang="en",
                                        transcript=lines, format_description=format_results, web_context=web_search_results)

pprint.pp(characters_output)

('- **誘導者の残響**: The Echo of the Inducer, a resurrected dark god from the '
 'abyss, referred to as the highest inducer. Engages in discussions about '
 'power and identity. [Narrative Focus]\n'
 '- **スロアン**: A character expressing disbelief and urgency, suggesting a sense '
 'of urgency and concern.\n'
 '- **シヴ・アラス**: Refers to the Echo of the Inducer as a brother and is deeply '
 'involved in the dialogue about betrayal and power struggles. Shows loyalty '
 'and determination.\n'
 '- **サバスン**: Engages in conversations about truth and deception, challenging '
 'the beliefs and actions of others. Appears strategic and manipulative.\n'
 '- **エリス・モーン**: Provides commentary on the situation, pointing out flaws and '
 'offering a perspective on the unfolding events. Seems to hold a position of '
 'insight or observation.')


## Summarize

In [11]:
from src.logic.context import summarize_scene

In [12]:
summary_output = summarize_scene(llm, input_lang="ja", output_lang="en",
                                 transcript=lines, format_description=format_results, 
                                 web_context=web_search_results, character_list=characters_output)

pprint.pp(summary_output)

('In this scene, the Echo of the Inducer, a resurrected dark god, confronts '
 'several characters, including Sloane, Xivu Arath, Savathun, and Eris Morn, '
 'in a tense and dynamic exchange. The Echo grapples with its identity and the '
 'implications of its resurrection, questioning its own existence and the '
 'betrayal it feels. Xivu Arath shows loyalty to the Echo, urging it to '
 "reclaim its throne through vengeance, while Savathun challenges the Echo's "
 'beliefs, revealing that the gods were manipulated by a greater force. '
 "Savathun's manipulative nature is evident as she tries to sway the Echo and "
 'others to her perspective, suggesting a path of independence from divine '
 'influences. Eris Morn provides commentary on the unfolding events, '
 "highlighting the flaws in Oryx's logic and the resulting chaos. The scene is "
 'filled with emotional tension, as characters confront their past, their '
 'loyalties, and the shifting power dynamics among them. Ultimately, the E

## Tone

In [13]:
from src.logic.context import determine_tone

In [14]:
tone_output = determine_tone(llm, input_lang='ja', output_lang='en',
                             transcript=lines, format_description=format_results,
                             web_context=web_search_results)

pprint.pp(tone_output)

('The overall tone of the scene is serious and intense, with a focus on themes '
 'of power, betrayal, and existential reflection. The speech is formal and '
 'direct, with characters often using commanding and assertive language, '
 'reflecting their high stakes and complex relationships.')
